In [1]:
# Q1: Running Elastic

In [7]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [10]:
import minsearch

# Indexing the document
index = minsearch.Index(
    text_fields=["question","text","section"],
    keyword_fields=["course"]
)

In [11]:
from elasticsearch8 import Elasticsearch

In [12]:
es_client = Elasticsearch('http://localhost:9200')

In [13]:
es_client.info()

ObjectApiResponse({'name': 'b6c8b8d46627', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'klGDbvnHQlaT-VkUGrMzpQ', 'version': {'number': '8.17.6', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7', 'build_date': '2025-04-30T14:07:12.231372970Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [14]:
# Q2: Indexing the data

In [15]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [16]:
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:07<00:00, 121.49it/s]


In [18]:
# Q3: Searching

In [19]:
query = "How do execute a command on a Kubernetes pod?"

In [20]:
def elastic_search(query):

    search_query = {
        "size": 5,
        "query": {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["question^4", "text"],
                            "type": "best_fields"
                        }
                    },
                    # "filter": {
                    #     "term": {
                    #         "course": "data-engineering-zoomcamp"
                    #     }
                    # }
                }
            }
        # }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_score'])

    return result_docs

In [21]:
elastic_search(query)

[44.50556, 35.433445, 33.70974, 33.2635, 32.589073]

In [22]:
# Q4: Filtering

In [23]:
query2 =  "How do copy a file to a Docker container?"

In [46]:
def elastic_search2(query):

    search_query = {
        "size": 3,
        "query": {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["question^4", "text"],
                            "type": "best_fields"
                        }
                    },
                    "filter": {
                        "term": {
                            "course": "machine-learning-zoomcamp"
                        }
                    }
                }
            }
        # }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [47]:
elastic_search2(query2)

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [48]:
# Q5: Building a prompt

In [49]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [50]:
q = "How do copy a file to a Docker container?"
search_results = elastic_search2(q)

In [51]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [52]:
context=""

for doc in search_results:
    context = context + context_template.format(question=doc['question'],text=doc['text']) + "\n\n"

In [53]:
prompt = prompt_template.format(question=q, context=context).strip()

In [54]:
prompt

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do copy a file to a Docker container?\n\nCONTEXT:\nQ: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container

In [55]:
len(prompt)

1446

In [60]:
# Q6: Tokens

In [61]:
import tiktoken

In [62]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [63]:
encoding.decode_single_token_bytes(63842)

b"You're"

In [64]:
len(encoding.encode(prompt))

320

In [65]:
# Generating the answer

In [66]:
from openai import OpenAI

In [67]:
client = OpenAI()

In [74]:
def llm(prompt):
    response=client.chat.completions.create(
        model = 'gpt-4o',
        messages=[{"role": "user", "content":prompt}]
    )
    return response.choices[0].message.content

In [81]:
def search(query):
    boost = {'question':4.0}

    results = index.search(
        query = query,
        filter_dict= {'course':'machine-learning-zoomcamp'},
        boost_dict=boost,
        num_results=3
        )
    return results

In [82]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()

    context=""

    for doc in search_results:
        context = context + context_template.format(question=doc['question'],text=doc['text']) + "\n\n"

    prompt = prompt_template.format(question=q, context=context).strip()
    return prompt

In [88]:
llm(prompt)

"You can copy files from your local machine into a Docker container using the `docker cp` command. Here's the basic syntax to copy a file or directory from your local machine into a running Docker container:\n\n```bash\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\n```"

In [ ]:
# Calculating the costs

In [92]:
# Cost for 1000 requests = $4.5
Cost = (1000*150*0.005/1000)+(1000*250*0.015/1000)

In [93]:
Cost

4.5